# Wind power calculation

In [73]:
from pathlib import Path
import math

import pandas as pd
import numpy as np
from scipy.interpolate import RegularGridInterpolator

In [2]:
# Directory where data files will be downloaded
cwd_path = Path.cwd()
data_path = cwd_path.parent.joinpath('data')
data_push_path = cwd_path.parent.joinpath('data_to_push')

In [3]:
# Read in latest turbine data
# Now only 30,642 after removing 3 turbines outside bounding box
df = pd.read_pickle(data_push_path / 'df_turbines_knn_blades_haversine_elevation.pkl')

## Wind power density (WPD) equation with extra terms
My equations are as follows:  

- `WPD = (1/2) * (air_density) * (area_blades_swept) * (wind_velocity**3) * (turbine_efficiency)`
- `P_net = WPD * turbine_efficiency - P_required`

#### Variable descriptions:
- `WPD` is the wind power density in watts/m^2 (W/m^2)
- Assume constant air density of 1.225 kg/m3
- `turbine_efficiency` is what % of the WPD is converted to electrical energy fed into the grid 
    - Betz limit (Cp) is 0.59 or 59%. This is the theoretical maximum efficiency converting wind power density (WPD) to electricity (assuming no other losses)
    - In reality, most turbines extract about 20-50% (fairly large range and depends on turbine's power curve)
    - Assume 50% converting WPD to rotational energy for the generator; here and further reduce this 
    - Turbine generator efficiency converting mechanical energy to electricity is typically around 0.8 to 0.9 (80-90%)
    - Lump this in with Betz limit to create overall efficiency factor
    - If WPD to generator efficiency is 50%, and generator efficiency is 80%, total turbine efficiency can be approximated like this:
        - 0.5 × 0.8 = 0.4 (40% overall efficiency)
- `P_required` is the electricity required by the wind turbine to operate (in watts)
    - "These turbines are equipped with more advanced systems such as yaw systems to orient the turbine towards the wind, advanced control systems, cooling systems for the generator, heating systems for de-icing, and sometimes hydraulic systems. The operational consumption for such turbines typically ranges from 10,000 to 50,000 Watts (10-50 kW) or 10-50 kWh for one hour."
- `P_net` is the total net power in watts fed to the grid

In [16]:
def wind_power_output_in_watts(area_swept_by_blades, wind_speed_hub_height, turbine_efficiency, power_required, air_density=1.225):
    """
    Calculate the net power output of a wind turbine.
    """
    # Betz coefficient
    betz_limit = 0.59
    # Wind power equation (gross power output)
    power = 0.5 * air_density * area_swept_by_blades * wind_speed_hub_height**3 * betz_limit * turbine_efficiency
    # Net power output after subtracting operational power
    power_net = power - power_required
    
    return power_net

In [17]:
df['area_blades'].mean()

5815.457705110633

In [18]:
# Area in meters
area_swept_by_blades = 5815
# Mean wind speed in m/s
wind_speed_hub_height = 15
# Typical turbine electrical conversion efficiency
turbine_efficiency = 0.90
# Estimated 2000 Watts required for operations over the hour
power_required = 2000

In [26]:
# Execute function to calculate electricity (in Watts) fed into grid over one hour
output_watts = wind_power_output_in_watts(area_swept_by_blades, wind_speed_hub_height, turbine_efficiency, power_required)
output_watts

6380989.210937501

In [25]:
# Divide by 1000 to get kW
output_kw = output_watts / 1_000
output_kw

6380.9892109375005

-----

### Calculate hourly net output in MWh for one turbine

#### Ideas:
- Combine Betz limit and turbine efficiency terms
    - Typical turbine is around 30-45% efficient (up to 50% during peak wind times)
- Use net nominal power value (Nettonennleistung) to calculate power_required
    - Turbines require around 1-3% of their net nominal power to operate every hour
    - These values from wind turbine dataset are in kW!
- How and where to include the typical turbine 'cut in wind speed', the minimum wind speed needed for turbine to start running?
    - This is typically 3-5 m/s

In [35]:
df['Nettonennleistung'].describe()

count    30642.000000
mean      1986.239546
std       1117.443776
min          0.120000
25%       1300.000000
50%       2000.000000
75%       2530.000000
max       8000.000000
Name: Nettonennleistung, dtype: float64

##### Function that calculates hourly net power output to grid in units MWh

In [70]:
def hourly_output_in_mwh(area_swept_by_blades, wind_speed_hub_height, net_nominal_power, air_density=1.225):
    """
    Calculate the net hourly power output for a turbine in MWh
    As this is power calculated for one hour, output is MWh
    Note: input variable net_nominal_power is in kW!
    Variable wind_speed_hub_height is calculated using ERA5 wind speed data
    """
    # Typical turbine efficiency
    turbine_efficiency = 0.40
    
    # Wind power equation to calculate gross power output in Watts for the hour (Power = WPD (wind power density) * efficiency)
    power = 0.5 * air_density * area_swept_by_blades * wind_speed_hub_height**3 * turbine_efficiency
    
    # Estimate power required in Watts using net nominal power of turbine in kW multiplied by percentage factor
    power_required = (net_nominal_power * 1000) * 0.01
    
    # Net power output in Watts after subtracting required operational power
    power_net = power - power_required
    
    # Convert Watts to MW
    power_net_mwh = power_net / 1_000_000
    
    # Calculated for one hour so unit is MWh
    return power_net_mwh

##### Test and validate the function for sensible output

In [71]:
# Area in meters
area_swept_by_blades = 5815
# Mean wind speed over the hour in m/s
wind_speed_hub_height = 15
# Net nominal power in KW
net_nominal_power = 2_000

In [72]:
# Execute function to calculate electricity (MWh) fed into grid over one hour
hourly_output_mwh = hourly_output_in_mwh(area_swept_by_blades, wind_speed_hub_height, net_nominal_power)
hourly_output_mwh

4.788278125000001

-----

## Xarray and `scipy.interpolate.RegularGridInterpolator`